In [1]:
import os
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from PIL import Image
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import train_test_split as tts
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained ViT model and feature extractor
model_name = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(model_name, num_labels=2)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [3]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Load dataset
dataset = ImageFolder('data/original-dataset', transform=transform)

In [4]:
train_dataset, test_dataset = tts(dataset, test_size=0.2)

c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
len(train_dataset), len(test_dataset)
dataset.class_to_idx

{'CANCER': 0, 'NON_CANCER': 1}

In [6]:
folder_path = "data/NON_CANCER(Augmented)"

image_files = [f for f in os.listdir(folder_path)]

augmented_images = []
for image_file in image_files:
    img_path = os.path.join(folder_path, image_file)
    img = Image.open(img_path)
    augmented_images.append((transform(img), 1))
    
train_dataset = train_dataset + augmented_images
train_dataset.pop(-1)
train_dataset.pop(-1)

(tensor([[[-0.9608, -0.9686, -0.9686,  ..., -0.9686, -0.9529, -0.9608],
          [-0.9608, -0.9608, -0.9608,  ..., -0.9608, -0.9608, -0.9686],
          [-0.9608, -0.9608, -0.9608,  ..., -0.9529, -0.9608, -0.9529],
          ...,
          [-0.9529, -0.9608, -0.9608,  ..., -0.9608, -0.9529, -0.9451],
          [-0.9686, -0.9608, -0.9529,  ..., -0.9608, -0.9608, -0.9608],
          [-0.9686, -0.9529, -0.9608,  ..., -0.9608, -0.9608, -0.9686]],
 
         [[-0.9608, -0.9686, -0.9686,  ..., -0.9686, -0.9529, -0.9608],
          [-0.9608, -0.9608, -0.9608,  ..., -0.9608, -0.9608, -0.9686],
          [-0.9608, -0.9608, -0.9608,  ..., -0.9529, -0.9608, -0.9529],
          ...,
          [-0.9529, -0.9608, -0.9608,  ..., -0.9608, -0.9529, -0.9451],
          [-0.9686, -0.9608, -0.9529,  ..., -0.9608, -0.9608, -0.9608],
          [-0.9686, -0.9529, -0.9608,  ..., -0.9608, -0.9608, -0.9686]],
 
         [[-0.9608, -0.9686, -0.9686,  ..., -0.9686, -0.9529, -0.9608],
          [-0.9608, -0.9608,

In [7]:
len(train_dataset), len(augmented_images)

(800, 50)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# Define optimizer and loss function
optimizer = Adam(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()

# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            val_loss += criterion(outputs, labels).item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / len(val_loader.dataset)
    print(f'Epoch {epoch + 1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    f1 = f1_score(labels, preds, average='binary')
    
    file_handler = open(f"metrics/vision-model-metrics.txt", "a")
    file_handler.write(f"Model{epoch}: Accuracy ({val_accuracy}) | Precision ({precision}) | Recall ({recall}) | F1 ({f1})\n")
    file_handler.close()
            
    torch.save(model.state_dict(), os.path.join('vision-transformer-models', f'model{epoch}.pth'))

100%|██████████| 25/25 [05:21<00:00, 12.87s/it]


Epoch 1, Val Loss: 0.0073, Val Accuracy: 0.9202


100%|██████████| 25/25 [05:09<00:00, 12.37s/it]


Epoch 2, Val Loss: 0.0074, Val Accuracy: 0.9202


100%|██████████| 25/25 [05:06<00:00, 12.24s/it]


...

Epoch 48, Val Loss: 0.0112, Val Accuracy: 0.9255


100%|██████████| 25/25 [04:54<00:00, 11.76s/it]


Epoch 49, Val Loss: 0.0113, Val Accuracy: 0.9255


100%|██████████| 25/25 [04:55<00:00, 11.82s/it]


Epoch 50, Val Loss: 0.0113, Val Accuracy: 0.9255
